In [9]:
!pip install pandas openpyxl fuzzywuzzy python-Levenshtein groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 4.7 MB/s eta 0:00:00


In [10]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from openpyxl import load_workbook
from groq import Groq

In [11]:
from google.colab import files

# Upload your files
uploaded = files.upload()

# Save file names
file_paths = list(uploaded.keys())
template_file = [f for f in file_paths if 'template' in f.lower()][0]
source_file = [f for f in file_paths if 'source' in f.lower()][0]

print(f"Template File: {template_file}")
print(f"Source File: {source_file}")

Saving source.xlsx to source (1).xlsx
Saving template.xlsx to template (1).xlsx
Template File: template (1).xlsx
Source File: source (1).xlsx


In [12]:
# Read all sheets from source.xlsx
source_sheets = pd.read_excel(source_file, sheet_name=None)

# Display available sheets
for sheet in source_sheets:
    print(f"\nSheet: {sheet}")
    print(source_sheets[sheet].head())


Sheet: agency_staff_costs
          Facility                                               Role  \
0  ACH Valley View                              â—¦ Registered nurses   
1  ACH Valley View   â—¦ Personal care workers / assistant in nursing   
2  ACH Valley View  â—¦ Diversional/Lifestyle/Recreation/Activitie...   
3    ACH Riverbank                              â—¦ Registered nurses   
4    ACH Riverbank   â—¦ Personal care workers / assistant in nursing   

   Cost_AUD QuarterStart QuarterEnd  
0     47696   2025-04-01 2025-06-30  
1     42298   2025-04-01 2025-06-30  
2     38210   2025-04-01 2025-06-30  
3     30189   2025-04-01 2025-06-30  
4     34110   2025-04-01 2025-06-30  

Sheet: bed_days
          Facility  OccupiedBedDays  AvailableBedDays QuarterStart QuarterEnd
0  ACH Valley View             6161              6291   2025-04-01 2025-06-30
1    ACH Riverbank             5435              5612   2025-04-01 2025-06-30
2  ACH Meadowfield             5810              5977  

In [13]:
# Load the template and extract headers
wb_template = load_workbook(template_file)
ws_template = wb_template.active

# Get header row (assumes headers are on first row)
template_headers = [cell.value for cell in ws_template[1]]
template_headers = [h for h in template_headers if h is not None]  # Remove empty cells

print("Template Headers:")
print(template_headers)

Template Headers:
[' Residential Aged Care Home Expenditure for the quarter 1 April to 30 June 2025']


In [17]:
# Replace with your Groq API key
GROQ_API_KEY = "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"

client = Groq(api_key=GROQ_API_KEY)

In [19]:
def ai_map_with_groq(template_headers, source_columns):
    prompt = f"""
You are given two lists:
- Template Headers: {template_headers}
- Source Columns: {source_columns}

Your task is to match each Template Header to the most relevant Source Column.
Return JSON like: {{ "header1": "column1", ... }}
If no good match exists, return null for that header.

Example:
{{
  "Total Employee Labour Costs - Direct Care": "Cost_AUD",
  "Occupied bed days": null
}}
"""

    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-8b-8192"
    )

    return response.choices[0].message.content

In [20]:
# Choose one sheet from source to test mapping
source_sheet_name = "employee_labour_costs"
source_df = source_sheets[source_sheet_name]

# Get source column names
source_columns = source_df.columns.tolist()
print("Source Columns:", source_columns)

# Ask Groq to map headers
mapping_response = ai_map_with_groq(template_headers, source_columns)
print("\nAI Mapping Response:\n", mapping_response)

Source Columns: ['Facility', 'Role', 'Cost_AUD', 'QuarterStart', 'QuarterEnd']

AI Mapping Response:
 Here is the Python code to solve the problem:

```python
template_headers = ['Residential Aged Care Home Expenditure for the quarter 1 April to 30 June 2025', 
                    'Total Employee Labour Costs - Direct Care', 
                    'Occupied bed days']

source_columns = ['Facility', 'Role', 'Cost_AUD', 'QuarterStart', 'QuarterEnd']

header_matches = {}

for header in template_headers:
    match = None
    for column in source_columns:
        if column in header:
            match = column
            break
    header_matches[header] = match

print(header_matches)
```

This code will iterate through each header in the template headers list and check if any of the source columns match substrings within the header. If it finds a match, it will store that match in the `header_matches` dictionary. If no match is found, it will store `None` for that header. The result will be 

In [22]:
import json

try:
    field_mapping = json.loads(mapping_response)
except json.JSONDecodeError:
    print("⚠️ Failed to parse AI response as JSON")
    field_mapping = {}

print("\nParsed Field Mapping:")
print(field_mapping)

⚠️ Failed to parse AI response as JSON

Parsed Field Mapping:
{}


In [23]:
def fuzzy_match_all(template_headers, source_columns):
    mapping = {}
    for header in template_headers:
        best_match, score = process.extractOne(header, source_columns, scorer=fuzz.token_sort_ratio)
        if score > 70:
            mapping[header] = best_match
        else:
            mapping[header] = None
    return mapping

# Combine or fallback
final_mapping = {}
for k, v in field_mapping.items():
    if v is not None:
        final_mapping[k] = v
    else:
        # Try fuzzy fallback
        matches = {k2: v2 for k2, v2 in fuzzy_match_all([k], source_columns).items()}
        final_mapping.update(matches)

print("\nFinal Mapping (AI + Fuzzy):")
print(final_mapping)


Final Mapping (AI + Fuzzy):
{}


In [24]:
# Reload template workbook to write data
wb_output = load_workbook(template_file)
ws_output = wb_output.active

# Assume Facility is in column A
facility_col_index = 1  # Column A

# Loop through rows starting from row 2
for row in ws_output.iter_rows(min_row=2):
    facility_cell = row[0]  # First column is Facility
    facility_name = facility_cell.value

    if facility_name and facility_name in source_df.values:
        for col_idx, header in enumerate(template_headers):
            src_col = final_mapping.get(header)
            if src_col:
                try:
                    value = source_df.loc[source_df["Facility"] == facility_name, src_col].values[0]
                    ws_output.cell(row=facility_cell.row, column=col_idx + 1, value=value)
                except Exception as e:
                    print(f"Failed to insert value for {header}: {e}")

# Save output
output_file = "populated_template.xlsx"
wb_output.save(output_file)
print(f"\n✅ Populated template saved as '{output_file}'")


✅ Populated template saved as 'populated_template.xlsx'


In [25]:
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>